### DATA COLLECTION

In [ ]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import copy
import csv
import os
from lib.rtls_slave import pixel_calculate, aoa_main, sort_data
import json


ENVIRONMENT = 'random'

# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:yaw>/<signed_int:roll>/<signed_int:pitch>', methods=['GET'])
def get_angle(yaw,roll,pitch):
    print(f"YAW: {yaw}, ROLL: {roll}, PITCH: {pitch}")
    global aoa
    global aoabefore
    
    print("Plane Angle: " + str(roll))
    aoa_result = aoa_main()
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(roll)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result

    with open(f"./data/aoa_data/{ENVIRONMENT}_{roll}.json",'w') as f:
        json.dump(aoa_result, f)
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    # ANGLE SORTING TECHNIQUES
    aoabefore_ = sort_data(aoabefore,aoa_bias = -35)
    aoa_ = sort_data(aoa,aoa_bias = -35)

    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    set_aoa = set(aoa_)
    set_aoabefore = set(aoabefore_)

    for slave in set_aoa.intersection(set_aoabefore):
        f = open(f"./results/pixels/{slave.replace(':','_')}.csv",'w',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        data_before = aoabefore_[slave]
        data_after = aoa_[slave]

        for i in range(len(data_before[0])):
            for j in range(len(data_after[0])):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], data_before[0][i], data_after[0][j])
                count = data_before[1][i]*data_after[1][j]
                if temp1 != -1:
                    tux = (temp1, temp2,count)
                    writer.writerow(tux)

        f.close()
                
        
        f.close()

    
    return jsonify({'state': 'success'})

In [ ]:
ENVIRONMENT = 'random'
app.run(host='0.0.0.0',port = 5000, debug=False)
# My Charger: 80:6F:B0:EE:AD:32 : Water Bottle.
# Ishrath Charger:  80:6F:B0:EE:AA:13 : Tea Bottle.

### POST POINT CALCULATION

In [2]:
from lib.rtls_slave import post_calculation

count = 0
with open("./data/weights/calculation_reference.txt", "r") as file:
    for line in file:
        count = count + 1
        ctnt = line.split('_')
        CASE = '_'.join(ctnt[:-2])
        L1 = int(ctnt[-2])
        L2 = int(ctnt[-1])
        print(f"CASE {count}: {CASE}")
        post_calculation(CASE,L1,L2,aoa_bias = -45)

CASE 1: test1
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[14.05882353 14.58823529 15.11764706 15.64705882 16.17647059 16.70588235
 17.23529412 17.76470588] 
	AoA After:[13.72972973 14.24324324 14.75675676 15.27027027 15.78378378 16.2972973
 16.81081081]
	Center: (837.153862367154, 1910.3885587241434)

CASE 2: test2
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[-4.  -3.4 -2.8 -2.2 -1.6 -1. ] 
	AoA After:[-4.  -3.4 -2.8 -2.2 -1.6 -1. ]
	Center: (2488.410007683572, 1868.83928152116)

CASE 3: test3
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[10.15384615 10.69230769 11.23076923 11.76923077 12.30769231 12.84615385
 13.38461538] 
	AoA After:[10.         10.66666667 11.33333333 12.        ]
	Center: (1201.6193284701399, 1898.5460709464414)

CASE 4: test4
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[-15.36363636 -14.81818182 -14.27272727 -13.72727273 -13.18181818
 -12.63636364 -12.09090909 -11.54545455] 
	AoA A

c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np

Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[15.72131148 16.2295082  16.73770492 17.24590164 17.75409836 18.26229508
 18.7704918  19.27868852] 
	AoA After:[4.79012346 5.2962963  5.80246914 6.30864198 6.81481481 7.32098765
 7.82716049 8.33333333]
	Center: (-1, -1)

CASE 11: test11
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[17.13333333 17.66666667 18.2        18.73333333 19.26666667 19.8
 20.33333333] 
	AoA After:[11.09090909 11.63636364 12.18181818 12.72727273 13.27272727 13.81818182
 14.36363636]
	Center: (1223.277057989054, 2717.4040193592064)

CASE 12: test12
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-6.4        -5.86666667 -5.33333333 -4.8        -4.26666667 -3.73333333
 -3.2        -2.66666667] 
	AoA After:[-9.8        -9.26666667 -8.73333333 -8.2        -7.66666667 -7.13333333
 -6.6        -6.06666667]
	Center: (2325.8219144755376, 1595.1832257410106)

CASE 13: test13
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:

c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\Users\sanji\.pyenv\pyenv-win\versions\3.8.8\lib\site-packages\sklearn\neighbors\_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (19). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\Users\sanji\.pyenv\pyenv-win\versions\3.8.8\lib\site-packages\sklearn\neighbors\_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (6). n_neighbors will be set to (n_samples - 1) for e

Rotation Angle: 64 --> 72

Slave :F8:8A:5E:2D:89:EA 
	AoA Before:[-15.93103448 -15.42528736 -14.91954023 -14.4137931  -13.90804598
 -13.40229885 -12.89655172 -12.3908046 ] 
	AoA After:[-17.35294118 -16.82352941 -16.29411765 -15.76470588 -15.23529412
 -14.70588235 -14.17647059]
	Center: (3071.996432662374, 1572.5052910234062)

CASE 19: test19
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-8.  -7.4 -6.8 -6.2 -5.6 -5. ] 
	AoA After:[-8.8  -8.28 -7.76 -7.24 -6.72 -6.2  -5.68 -5.16]
	Center: (2399.8541032276694, 1759.204749259877)

CASE 20: test20
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[25.55555556 26.11111111 26.66666667 27.22222222 27.77777778 28.33333333
 28.88888889] 
	AoA After:[20.  20.6 21.2 21.8 22.4 23. ]
	Center: (608.2932581204657, 3203.1241903474815)

CASE 21: test21
Rotation Angle: 52 --> 62

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[1.         1.66666667 2.33333333 3.        ] 
	AoA After:[-1.  -0.4  0.2  0.8  1.4  2. ]
	Center: (-1, 

...

c:\Users\sanji\.pyenv\pyenv-win\versions\3.8.8\lib\site-packages\sklearn\neighbors\_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (2). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(


Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-12.88888889 -12.33333333 -11.77777778 -11.22222222 -10.66666667
 -10.11111111  -9.55555556] 
	AoA After:[-10.84 -10.32  -9.8   -9.28  -8.76  -8.24  -7.72  -7.2 ]
	Center: (2645.5960177469556, 2040.0306012433286)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[28.61538462 29.15384615 29.69230769 30.23076923 30.76923077 31.30769231
 31.84615385] 
	AoA After:[27.63636364 28.18181818 28.72727273 29.27272727 29.81818182 30.36363636
 30.90909091]
	Center: (-421.6165980800906, 2173.864898504674)

CASE 107: test45
Rotation Angle: 50 --> 60

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-26.23809524 -25.71428571 -25.19047619 -24.66666667 -24.14285714
 -23.61904762 -23.0952381 ] 
	AoA After:[-25.82352941 -25.29411765 -24.76470588 -24.23529412 -23.70588235
 -23.17647059 -22.64705882 -22.11764706]
	Center: (3901.2977469217676, 2029.5121198396362)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[25.55555556 26.11111111 26.66666667 27.22222222 27.77777778 28.33333333
 28.88888889] 
	

c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))


Rotation Angle: 60 --> 70

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-25.90909091 -25.36363636 -24.81818182 -24.27272727 -23.72727273
 -23.18181818 -22.63636364 -22.09090909] 
	AoA After:[-27.33333333 -26.8        -26.26666667 -25.73333333 -25.2
 -24.66666667 -24.13333333]
	Center: (3911.591514788464, 1540.4102994266234)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[20.18918919 20.7027027  21.21621622 21.72972973 22.24324324 22.75675676
 23.27027027] 
	AoA After:[20.7037037  21.22222222 21.74074074 22.25925926 22.77777778 23.2962963
 23.81481481 24.33333333]
	Center: (312.2172282909273, 1747.655447271135)

CASE 111: test47
Rotation Angle: 60 --> 70

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-23.86666667 -23.33333333 -22.8        -22.26666667 -21.73333333
 -21.2        -20.66666667] 
	AoA After:[-24.86666667 -24.33333333 -23.8        -23.26666667 -22.73333333
 -22.2        -21.66666667 -21.13333333]
	Center: (3731.4208820973226, 1733.8676410127537)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[-2.8666666

c:\INTERN\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))


### VISUALIZING THE ANGLE

In [ ]:
from lib.rtls_slave import visualize_aoa_spread

CASE = 'three_down_indoor_i'
level = 74
visualize_aoa_spread(CASE, level)

In [ ]:
from lib.rtls_slave import visualize_aoa_turn_spread

CASE = 'two_top_indoor'
level1 = 68
level2 = 76
vis_bias = 0

visualize_aoa_turn_spread(CASE, level1, level2, vis_bias)

### RAW DATA VISUALIZATION

In [ ]:
import matplotlib.pyplot as plt
import json

def visualize_aoa_spread(CASE, level, num_bins = 100):

    with open(f'./data/aoa_data/{CASE}_{level}.json', 'r') as f:
        aoa_data = json.load(f)

    # Create a figure and axis object
    fig, ax = plt.subplots()

    # Set the x and y labels and title
    ax.set_xlabel("Angle of Arrival")
    ax.set_ylabel("# of Occurance")
    ax.set_title("Histogram of Angle of Arrival")

    # Set the number of bins in the histogram

    # Loop over each slave and plot its angle of arrival data as a histogram
    for slave, data in aoa_data.items():
        ax.hist(data, bins=num_bins, alpha=0.5, label=slave)

    # Add a legend to the plot
    ax.legend()

    # Show the plot
    plt.show()

    if len(aoa_data.keys())>1:
        # Set up the figure with subplots
        fig, axs = plt.subplots(nrows=1, ncols=len(aoa_data.keys()), figsize=(10, 5))

        # Iterate over each slave and plot a histogram of its angle of arrival data
        for i, slave in enumerate(aoa_data.keys()):
            axs[i].hist(aoa_data[slave], bins=num_bins,color='lightblue')  # adjust bins as needed
            axs[i].set_title(slave)
            axs[i].set_xlabel("Angle of Arrival")
            axs[i].set_ylabel("# of Occurance")

        # Adjust the spacing between subplots and display the figure
        plt.tight_layout()

        plt.show()
        
        return "SUCESS"
    

visualize_aoa_spread('one_top_indoor',70,100)